In [16]:
import numpy as np
Aeq = np.load('Aeq.npy')
beq = np.load('beq.npy')
c = np.load('c.npy')

matlab_result = np.array([0.0960, 0, 0.0784, 0, 0.0110, 0.0875, 0.0911, 0.0172, 0.0804, 0, 0.0233, 0.1085, 0, 0.1469, 0.0806, 0.0038, 0.0008, 0.0777, 0, 0.0969]).reshape((10, 2))



In [18]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, PULP_CBC_CMD

# Define the LP problem
prob = LpProblem("My_LP_Problem", LpMinimize)

# Define the decision variables
x = [LpVariable(f"x{i}", lowBound=0) for i in range(len(c))]

# Add the objective function
prob += lpSum(c[i] * x[i] for i in range(len(c)))

# Add the equality constraints
for i in range(len(beq)):
    prob += lpSum(Aeq[i][j] * x[j] for j in range(len(c))) == beq[i]

# Solve the problem using CBC solver
prob.solve(PULP_CBC_CMD(msg=False))

# Get the results
pulp_results = [var.varValue for var in x]

# Calculate the Euclidean distance to the MATLAB result
euclidean_distance_pulp = np.linalg.norm(np.array(pulp_results).reshape((10, 2)) - matlab_result)


In [20]:
matlab_result

array([[0.096 , 0.    ],
       [0.0784, 0.    ],
       [0.011 , 0.0875],
       [0.0911, 0.0172],
       [0.0804, 0.    ],
       [0.0233, 0.1085],
       [0.    , 0.1469],
       [0.0806, 0.0038],
       [0.0008, 0.0777],
       [0.    , 0.0969]])

In [10]:
from gurobipy import Model, GRB

# Create a new model
m = Model("mip1")

# Create variables
x = m.addVars(len(c), lb=0, vtype=GRB.CONTINUOUS, name="x")

# Set objective
obj = 0
for i in range(len(c)):
    obj += c[i] * x[i]
m.setObjective(obj, GRB.MINIMIZE)

# Add equality constraint
for i in range(len(beq)):
    constr = 0
    for j in range(len(c)):
        constr += Aeq[i, j] * x[j]
    m.addConstr(constr == beq[i])


# Optimize
m.optimize()

# Get the results
gurobi_results = [x[i].x for i in range(len(c))]

# Calculate the Euclidean distance to the MATLAB result
euclidean_distance_gurobi = np.linalg.norm(np.array(gurobi_results) - matlab_result)


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i9-13980HX, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 21 rows, 20 columns and 251 nonzeros
Model fingerprint: 0xe1732f24
Coefficient statistics:
  Matrix range     [9e-04, 1e+00]
  Objective range  [7e-03, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 7 rows and 6 columns
Presolve time: 0.01s
Presolved: 14 rows, 14 columns, 162 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.9453800e-03   1.599033e+00   0.000000e+00      0s
      13    7.2644785e-01   0.000000e+00   0.000000e+00      0s

Solved in 13 iterations and 0.01 seconds (0.00 work units)
Optimal objective  7.264478451e-01


In [12]:
euclidean_distance_gurobi

0.00010146152964871253